# Matrizes, Arrays, Tensores

## Referências

- Documentação oficial de Tensores do PyTorch
    http://pytorch.org/docs/master/tensors.html
- PyTorch para usuários NumPy:
    https://github.com/torch/torch7/wiki/Torch-for-Numpy-users

## NumPy array

In [0]:
import numpy as np

In [4]:
a = np.array([[2., 8., 3.],
              [0.,-1., 5.]])
a

array([[ 2.,  8.,  3.],
       [ 0., -1.,  5.]])

In [5]:
a.shape

(2, 3)

In [6]:
a.dtype

dtype('float64')

## PyTorch tensor

Os tensores do PyTorch só podem ser float, float32 ou float64

In [2]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x57b60000 @  0x7f156217a2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import torch

In [11]:
!ls ../

bin   content  dev  home  lib32  media	opt   root  sbin  swift  tmp	usr
boot  datalab  etc  lib   lib64  mnt	proc  run   srv   sys	 tools	var


### Convertendo NumPy array para tensor PyTorch

In [12]:
b = torch.Tensor(np.zeros((3,4)))
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

### Criando arrays e tensores constantes

In [13]:
c = np.ones((2,4)); c

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [14]:
d = torch.ones((2,4)); d

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])

### Criando arrays e tensores aleatórios

In [15]:
e = np.random.rand(2,4); e

array([[0.49642624, 0.983208  , 0.50910409, 0.49430034],
       [0.9002293 , 0.17151609, 0.20783737, 0.55555196]])

In [16]:
f = torch.rand(2,4); f

tensor([[0.7657, 0.0856, 0.2247, 0.2366],
        [0.7977, 0.0980, 0.7158, 0.8644]])

### Arrays aleatórios com semente, para reproduzir mesma sequência pseudoaleatória

In [17]:
np.random.seed(1234)
e = np.random.rand(2,4);e

array([[0.19151945, 0.62210877, 0.43772774, 0.78535858],
       [0.77997581, 0.27259261, 0.27646426, 0.80187218]])

In [18]:
torch.manual_seed(1234)
f = torch.rand(2,4); f

tensor([[0.0290, 0.4019, 0.2598, 0.3666],
        [0.0583, 0.7006, 0.0518, 0.4681]])

### Torch seed is different for GPU

In [4]:
import torch
if torch.cuda.is_available():
    torch.cuda.torch.manual_seed(1234)
    g = torch.cuda.torch.rand(2,4)
    print(g)
else:
    print("sem gpu")

tensor([[0.0290, 0.4019, 0.2598, 0.3666],
        [0.0583, 0.7006, 0.0518, 0.4681]])


## Conversões entre NumPy e Tensores PyTorch

### NumPy para Tensor PyTorch utilizando `.from_numpy()` - CUIDADO

Não são todos os tipos de elementos do array NumPy que podem ser convertidos 
para tensores PyTorch. Abaixo é um programa que cria uma tabela de equivalencias
entre os tipos do NumPy e os tipos do Tensor PyTorch:

In [6]:
import numpy as np
import pandas as pd 
dtypes = [np.uint8, np.int32, np.int64, np.float32, np.float64, np.double]
table = np.empty((2, len(dtypes)),dtype=np.object)
for i,t in enumerate(dtypes):
    a = np.array([1],dtype=t)
    ta = torch.from_numpy(a)
    table[0,i] = a.dtype.name
    table[1,i] = type(ta).__name__
pd.DataFrame(table)

,0,1,2,3,4,5
0,uint8,int32,int64,float32,float64,float64
1,Tensor,Tensor,Tensor,Tensor,Tensor,Tensor


### NumPy para Tensor utilizando `torch.FloatTensor()` - método recomendado

Existe uma cuidado importante a ser tomado na transformação de matrizes do NumPy para tensores PyTorch pois as funções de rede neurais do PyTorch utilizam o tipo FloatTensor e o NumPy utiliza como default o tipo float64, o que faz uma conversão automática para DoubleTensor do PyTorch e consequentemente gerando um erro.
A recomendação é utilizar o `torch.FloatTensor` para converter NumPy para tensores PyTorch:

In [7]:
a = np.ones((2,5))
a_t = torch.FloatTensor(a)
a_t

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

### Tensor PyTorch para array NumPy

In [8]:
ta = torch.ones(2,3)
ta

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [9]:
a = ta.numpy()
a

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

## Tensor na CPU e na GPU

In [10]:
ta_cpu = torch.ones(2,3); ta_cpu

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [11]:
if torch.cuda.is_available():
    ta_gpu = ta_cpu.cuda()
    print(ta_gpu)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


## Operações em tensores

### criação de tensor e visualização do seu shape

In [12]:
a = torch.eye(4); a

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [13]:
a.size()

torch.Size([4, 4])

### Reshape é feito com `view` em PyTorch

In [14]:
b = a.view(2,8); b

tensor([[1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1.]])

Aqui é um exemplo criando um tensor unidimensional sequencial de 0 a 23 e em seguida uma reshape para
que o tensor fique com 4 linhas e 6 colunas

In [18]:
a = torch.arange(0,24).view(4,6);a

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17],
        [18, 19, 20, 21, 22, 23]])

### Adição elemento por elemento

#### usando operadores

In [19]:
c = a + a; c

tensor([[ 0,  2,  4,  6,  8, 10],
        [12, 14, 16, 18, 20, 22],
        [24, 26, 28, 30, 32, 34],
        [36, 38, 40, 42, 44, 46]])

In [20]:
d = a - c ; d

tensor([[  0,  -1,  -2,  -3,  -4,  -5],
        [ -6,  -7,  -8,  -9, -10, -11],
        [-12, -13, -14, -15, -16, -17],
        [-18, -19, -20, -21, -22, -23]])

#### forma funcional

In [21]:
d = a.sub(c); d

tensor([[  0,  -1,  -2,  -3,  -4,  -5],
        [ -6,  -7,  -8,  -9, -10, -11],
        [-12, -13, -14, -15, -16, -17],
        [-18, -19, -20, -21, -22, -23]])

#### Operação in-place

In [22]:
a.sub_(c); a

tensor([[  0,  -1,  -2,  -3,  -4,  -5],
        [ -6,  -7,  -8,  -9, -10, -11],
        [-12, -13, -14, -15, -16, -17],
        [-18, -19, -20, -21, -22, -23]])

### Multiplicação elemento por elemento

In [23]:
d = a * c; d 

tensor([[    0,    -2,    -8,   -18,   -32,   -50],
        [  -72,   -98,  -128,  -162,  -200,  -242],
        [ -288,  -338,  -392,  -450,  -512,  -578],
        [ -648,  -722,  -800,  -882,  -968, -1058]])

In [24]:
d = a.mul(c); d

tensor([[    0,    -2,    -8,   -18,   -32,   -50],
        [  -72,   -98,  -128,  -162,  -200,  -242],
        [ -288,  -338,  -392,  -450,  -512,  -578],
        [ -648,  -722,  -800,  -882,  -968, -1058]])

In [25]:
a.mul_(c); a

tensor([[    0,    -2,    -8,   -18,   -32,   -50],
        [  -72,   -98,  -128,  -162,  -200,  -242],
        [ -288,  -338,  -392,  -450,  -512,  -578],
        [ -648,  -722,  -800,  -882,  -968, -1058]])

### Média em tensores

In [0]:
a = torch.arange(0.,24.).view(4,6)

In [30]:
u = a.mean(); u

tensor(11.5000)

In [31]:
uu = a.sum()/a.nelement(); uu

tensor(11.5000)

### Média com redução de eixo 

In [32]:
u_row = a.mean(dim=1); u_row

tensor([ 2.5000,  8.5000, 14.5000, 20.5000])

In [33]:
u_col = a.mean(dim=0); u_col

tensor([ 9., 10., 11., 12., 13., 14.])

### Desvio padrão

In [34]:
std = a.std(); std

tensor(7.0711)

In [35]:
std_row = a.std(dim=1); std_row

tensor([1.8708, 1.8708, 1.8708, 1.8708])

## Comparação speedup CPU e GPU

In [37]:
a_numpy_cpu = np.ones((1000,1000))
%timeit b = 2 * a_numpy_cpu
a_torch_cpu = torch.ones(1000,1000)
%timeit b = 2 * a_torch_cpu

1000 loops, best of 3: 1.38 ms per loop
1000 loops, best of 3: 696 µs per loop


In [38]:
if torch.cuda.is_available():
    a_torch_gpu = a_torch_cpu.cuda()
    %timeit b = 2 * a_torch_gpu

The slowest run took 8.37 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 55.1 µs per loop


Rodando o código abaixo na GTX1080: speedup de 15,5
- 888 µs ± 43.4 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
- 57.1 µs ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Rodando no macbook:
- numpy: 1000 loops, best of 3: 449 µs per loop
- torch: 1000 loops, best of 3: 1.6 ms per loop

In [40]:
%timeit b1 = a_numpy_cpu.mean()
%timeit b2 = a_torch_cpu.mean()
if torch.cuda.is_available():
    %timeit c = a_torch_gpu.mean()

The slowest run took 4.93 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 480 µs per loop
The slowest run took 4.40 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 146 µs per loop
1000 loops, best of 3: 218 µs per loop
